In [1]:
import torch
from torch import nn

In [2]:
import pandas as pd

df = pd.read_csv("./data/SMSSpamCollection", sep="\t", names=["type", "message"])
df["spam"] = df["type"] == "spam"
df.drop("type", axis=1, inplace=True)
df.head()

,message,spam
0,"Go until jurong point, crazy.. Available only ...",False
1,Ok lar... Joking wif u oni...,False
2,Free entry in 2 a wkly comp to win FA Cup fina...,True
3,U dun say so early hor... U c already then say...,False
4,"Nah I don't think he goes to usf, he lives aro...",False


In [3]:
print("SPAM messages:")
print(len(df[df["spam"] == True]))

SPAM messages:
747


In [4]:
print("SPAM messages:")
print(len(df[df["spam"] == False]))

SPAM messages:
4825


# Count vectorizer

In [5]:
df = pd.read_csv("./data/SMSSpamCollection", 
                 sep="\t", 
                 names=["type", "message"])

In [6]:
df["spam"] = df["type"] == "spam"
df.drop("type", axis=1, inplace=True)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer


In [8]:

cv = CountVectorizer(max_features=1000)
messages = cv.fit_transform(df["message"])
print(messages[0, :])
print(cv.get_feature_names_out()[888])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12 stored elements and shape (1, 1000)>
  Coords	Values
  (0, 347)	1
  (0, 886)	1
  (0, 656)	1
  (0, 202)	1
  (0, 88)	1
  (0, 608)	1
  (0, 427)	1
  (0, 359)	1
  (0, 972)	1
  (0, 828)	1
  (0, 356)	1
  (0, 920)	1
update


In [9]:

X = torch.tensor(messages.todense(), dtype=torch.float32)
y = torch.tensor(df["spam"], dtype=torch.float32)\
        .reshape((-1, 1))

model = nn.Linear(1000, 1)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for i in range(0, 10000):
    # Training pass
    optimizer.zero_grad()
    outputs = model(X)
    loss = loss_fn(outputs, y)
    loss.backward()
    optimizer.step()

    if i % 100 == 0: 
        print(loss)

model.eval()
with torch.no_grad():
    y_pred = model(X)
    print(y_pred)
    print(y_pred.min())
    print(y_pred.max())

tensor(0.1498, grad_fn=<MseLossBackward0>)
tensor(0.1258, grad_fn=<MseLossBackward0>)
tensor(0.1124, grad_fn=<MseLossBackward0>)
tensor(0.1037, grad_fn=<MseLossBackward0>)
tensor(0.0972, grad_fn=<MseLossBackward0>)
tensor(0.0919, grad_fn=<MseLossBackward0>)
tensor(0.0874, grad_fn=<MseLossBackward0>)
tensor(0.0834, grad_fn=<MseLossBackward0>)
tensor(0.0798, grad_fn=<MseLossBackward0>)
tensor(0.0766, grad_fn=<MseLossBackward0>)
tensor(0.0737, grad_fn=<MseLossBackward0>)
tensor(0.0710, grad_fn=<MseLossBackward0>)
tensor(0.0686, grad_fn=<MseLossBackward0>)
tensor(0.0664, grad_fn=<MseLossBackward0>)
tensor(0.0644, grad_fn=<MseLossBackward0>)
tensor(0.0626, grad_fn=<MseLossBackward0>)
tensor(0.0609, grad_fn=<MseLossBackward0>)
tensor(0.0593, grad_fn=<MseLossBackward0>)
tensor(0.0579, grad_fn=<MseLossBackward0>)
tensor(0.0565, grad_fn=<MseLossBackward0>)
tensor(0.0553, grad_fn=<MseLossBackward0>)
tensor(0.0541, grad_fn=<MseLossBackward0>)
tensor(0.0530, grad_fn=<MseLossBackward0>)
tensor(0.05

## BCE with Logits

In [10]:

model = nn.Linear(1000, 1)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

for i in range(0, 10000):
    # Training pass
    optimizer.zero_grad()
    outputs = model(X)
    loss = loss_fn(outputs, y)
    loss.backward()
    optimizer.step()

    if i % 1000 == 0: 
        print(loss)

# evaluate the model
model.eval()
with torch.no_grad():
    y_pred = (nn.functional.sigmoid(model(X)) > 0.5).type(torch.float32)

    # Accuracy
    accuracy = (y_pred == y).type(torch.float32).mean()

    # Sensitivity (recall for positives)
    sensitivity = (y_pred[y == 1] == y[y == 1]).type(torch.float32).mean()

    # Specificity (recall for negatives)
    specificity = (y_pred[y == 0] == y[y == 0]).type(torch.float32).mean()

    # Precision (of predicted positives, how many are correct)
    precision = (y_pred[y_pred == 1] == y[y_pred == 1]).type(torch.float32).mean()

    # Print nicely
    print(f"accuracy: {accuracy:.4f}")
    print(f"sensitivity: {sensitivity:.4f}")
    print(f"specificity: {specificity:.4f}")
    print(f"precision: {precision:.4f}")

    print("precision:" (y_pred[y_pred == 1] == y[y_pred == 1]).type(torch.float32).mean()) 

tensor(0.6968, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


<>:39: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:39: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?


tensor(0.2251, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1640, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1366, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1205, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1097, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1018, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0957, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0908, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0867, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


C:\Users\ibrah\AppData\Local\Temp\ipykernel_21984\4169624178.py:39: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  print("precision:" (y_pred[y_pred == 1] == y[y_pred == 1]).type(torch.float32).mean())


accuracy: 0.9772
sensitivity: 0.8514
specificity: 0.9967
precision: 0.9755


TypeError: 'str' object is not callable

## Training

In [11]:
import sys
import torch
from torch import nn
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv("./data/SMSSpamCollection", 
                 sep="\t", 
                 names=["type", "message"])

df["spam"] = df["type"] == "spam"
df.drop("type", axis=1, inplace=True)

df_train = df.sample(frac=0.8, random_state=0)
df_val = df.drop(index=df_train.index)

cv = CountVectorizer(max_features=5000)
messages_train = cv.fit_transform(df_train["message"])
messages_val = cv.transform(df_val["message"])

X_train = torch.tensor(messages_train.todense(), dtype=torch.float32)
y_train = torch.tensor(df_train["spam"].values, dtype=torch.float32)\
        .reshape((-1, 1))

X_val = torch.tensor(messages_val.todense(), dtype=torch.float32)
y_val = torch.tensor(df_val["spam"].values, dtype=torch.float32)\
        .reshape((-1, 1))

model = nn.Linear(5000, 1)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

for i in range(0, 10000):
    # Training pass
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = loss_fn(outputs, y_train)
    loss.backward()
    optimizer.step()

    if i % 1000 == 0: 
        print(loss)

def evaluate_model(X, y): 
    model.eval()
    with torch.no_grad():
        y_pred = nn.functional.sigmoid(model(X)) > 0.25
        print("accuracy:", (y_pred == y)\
            .type(torch.float32).mean())
        
        print("sensitivity:", (y_pred[y == 1] == y[y == 1])\
            .type(torch.float32).mean())
        
        print("specificity:", (y_pred[y == 0] == y[y == 0])\
            .type(torch.float32).mean())

        print("precision:", (y_pred[y_pred == 1] == y[y_pred == 1])\
            .type(torch.float32).mean()) 
        
print("Evaluating on the training data")
evaluate_model(X_train, y_train)

print("Evaluating on the validation data")
evaluate_model(X_val, y_val)

   

tensor(0.6942, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.2218, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1604, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1325, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1159, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1047, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0964, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0899, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0847, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0803, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Evaluating on the training data
accuracy: tensor(0.9809)
sensitivity: tensor(0.9293)
specificity: tensor(0.9891)
precision: tensor(0.9308)
Evaluating on the validation data
accuracy: tensor(0.9767)
sensitivity: tensor(0.9209)
specificity: tensor(0.9846)
precision: tensor(0.8951)


## Test

In [12]:

custom_messages = cv.transform([
    "We have release a new product, do you want to buy it?", 
    "Winner! Great deal, call us to get this product for free",
    "Tomorrow is my birthday, do you come to the party?"
])

X_custom = torch.tensor(custom_messages.todense(), dtype=torch.float32)

model.eval()
with torch.no_grad():
    pred = nn.functional.sigmoid(model(X_custom))
    print(pred)

tensor([[0.0517],
        [0.5956],
        [0.0130]])
